In [1]:
import torch
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision.transforms.functional as FT
from tqdm import tqdm
from torch.utils.data import DataLoader

In [2]:
# ============
# Dataset load
# ============
%run /content/drive/MyDrive/Colab\ Notebooks/YOLOv1/dataset.ipynb

Output hidden; open in https://colab.research.google.com to view.

In [3]:
# ==========
# LOSS Class
# ==========
%run /content/drive/MyDrive/Colab\ Notebooks/YOLOv1/loss.ipynb

In [4]:
# ==========
# YOLO Model
# ==========
%run /content/drive/MyDrive/Colab\ Notebooks/YOLOv1/models.ipynb

torch.Size([4, 1470])


In [5]:
# ==========
# Utils load
# ==========
%run /content/drive/MyDrive/Colab\ Notebooks/YOLOv1/utils.ipynb

In [6]:
# HYPER PARAMETERS
LEARNING_RATE = 2e-5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 32
WEIGHT_DECAY = 0
EPOCHS = 100

In [7]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [8]:
def train_fn(model, dataloader, optimizer, loss_fn):
  mean_loss = []
  for i, (X, y) in enumerate(dataloader):
    X, y = X.to(DEVICE), y.to(DEVICE)
    out = model(X)
    loss = loss_fn(out, y)
    mean_loss.append(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  return sum(mean_loss) / len(mean_loss)

In [13]:
# ==========
# TEST PHASE
# ==========
def test_fn(model, dataloader):
  ## step1. mAP check
  pred_boxes, target_boxes = get_bboxes(
      dataloader, model, iou_threshold=0.5, threshold=0.4
  )
  mean_avg_prec = mean_average_precision(
      pred_boxes, target_boxes, iou_threshold=0.5, box_format="midpoint"
  )
  print(f"Train mAP : {mean_avg_prec}")
  ## step2. showing bboxes
  x, y = next(iter(dataloader))
  x, y = x.to(DEVICE), y.to(DEVICE)
  bboxes = cellboxes_to_boxes(model(x)) # (batch, S * S, 6)
  for index in range(x.shape[0]):
    bbox = non_max_suppression(bboxes[index], iou_threshold=0.5, threshold=0.4, box_format="midpoint")
    plot_image(x[index].to("cpu"), bbox)

In [14]:
x, y = next(iter(val_dataloader))
x[0].shape

torch.Size([3, 448, 448])

In [15]:
def main():
  model = Yolov1(split_size=7, num_boxes=2, num_classes=20).to(DEVICE)
  optimizer = optim.Adam(
      model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY
  )
  loss_fn = YoloLoss()

  train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
  val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
  
  mean_losses = []
  for epoch in range(EPOCHS):
    pred_boxes, target_boxes = get_bboxes(
        train_dataloader, model, iou_threshold=0.5, threshold=0.4
    )
    mean_avg_prec = mean_average_precision(
        pred_boxes, target_boxes, iou_threshold=0.5, box_format="midpoint"
    )
    print(f"Train mAP: {mean_avg_prec}")
    if (epoch + 1) % 10 == 0:
      test_fn(model, val_dataloader)
    ##
    mean_loss = train_fn(model, train_dataloader, optimizer, loss_fn)
    mean_losses.append(mean_loss)
    print(f"{epoch+1:02} : Mean loss was {mean_loss:.4f}")
  plt.plot(mean_losses)


In [ ]:
main()

Output hidden; open in https://colab.research.google.com to view.